## import utilities

In [24]:
import torch
from torch import nn as nn
from torch import tensor
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


## Create Model

In [27]:
class MachineFailure(nn.Module):
    def __init__(self):
        super(MachineFailure, self).__init__()
        self.linear1 = nn.Linear(9, 7)
        self.linear2 = nn.Linear(7, 4)
        self.linear3 = nn.Linear(4, 1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        x = self.activation(x)
        return x

machine_data_model = MachineFailure()

## Load CSV data into pandas

In [28]:
data = pd.read_csv("./machine_failure.csv")
data.head()

,footfall,tempMode,AQ,USS,CS,VOC,RP,IP,Temperature,fail
0,0,7,7,1,6,6,36,3,1,1
1,190,1,3,3,5,1,20,4,1,0
2,31,7,2,2,6,1,24,6,1,0
3,83,4,3,4,5,1,28,6,1,0
4,640,7,5,6,4,0,68,6,1,0


## Drop the Fail feature

In [29]:
data.fillna(data.mean(), inplace=True)
features = data.drop("fail", axis=1)



## Create Label

In [30]:
labels = data["fail"]


## implement standard scaler

In [33]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)


[[-0.28315338  1.22289631  1.86049689 ... -0.67278806 -0.97950413
  -2.56740712]
 [-0.10755803 -1.01941016 -0.92177489 ... -1.64754019 -0.35389391
  -2.56740712]
 [-0.25450362  1.22289631 -1.61734284 ... -1.40385216  0.89732651
  -2.56740712]
 ...
 [-0.28315338 -0.27197467  1.16492894 ... -0.24633401  0.89732651
   1.28414704]
 [-0.28315338  0.84917856  1.16492894 ... -0.06356798  1.52293673
   1.28414704]
 [-0.26651803  1.22289631 -0.22620695 ...  0.85026213  1.52293673
   1.28414704]]


## Set up Training Features

In [65]:
training_features, testing_features, training_labels, testing_labels = train_test_split(scaled_features, labels, train_size=0.2)
training_features = tensor(training_features, dtype=torch.float32)
training_labels = tensor(training_labels.values, dtype=torch.float32)
testing_features = tensor(testing_features, dtype=torch.float32)
testing_labels = tensor(testing_labels.values, dtype=torch.float32)
print(repr(training_labels))
print(testing_labels.shape)
print(training_labels.shape)

tensor([0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
        1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
        0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
        0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1.,
        0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0.,
        1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0.,
        0., 1., 0., 0., 1., 1., 0., 0.])
torch.Size([756])
torch.Size([188])


## Create Datasets from Tensors

In [59]:
from torch.utils.data import DataLoader, TensorDataset

training_dataset = TensorDataset(training_features, training_labels)
testing_dataset = TensorDataset(testing_features, testing_labels)

## load data

In [63]:
training_loader = DataLoader(training_dataset, batch_size=32, shuffle=True)
testing_loader = DataLoader(testing_dataset, batch_size=32, shuffle=False)

## Start Training loop

In [72]:
from torch import optim

cryterion = nn.BCELoss()
optimizer = optim.Adam(machine_data_model.parameters(), lr=0.01)
machine_data_model.train()
num_of_epochs = 2000
for epoch in range(num_of_epochs):
    for features, labels in training_loader:
        optimizer.zero_grad()
        output = machine_data_model(features)
        loss = cryterion(output, labels.view(-1, 1))
        loss.backward()
        optimizer.step
    # if epoch % 100 == 0:
    print(f'Epoch {epoch+1}/{num_of_epochs}, Loss: {loss.item()}')

Epoch 1/2000, Loss: 0.7012659311294556
Epoch 2/2000, Loss: 0.7372382879257202
Epoch 3/2000, Loss: 0.7065082788467407
Epoch 4/2000, Loss: 0.7562891840934753
Epoch 5/2000, Loss: 0.6628912687301636
Epoch 6/2000, Loss: 0.6940415501594543
Epoch 7/2000, Loss: 0.6826602220535278
Epoch 8/2000, Loss: 0.7321369051933289
Epoch 9/2000, Loss: 0.6829854249954224
Epoch 10/2000, Loss: 0.7390893697738647
Epoch 11/2000, Loss: 0.7307764291763306
Epoch 12/2000, Loss: 0.6747025847434998
Epoch 13/2000, Loss: 0.6455675959587097
Epoch 14/2000, Loss: 0.7425804138183594
Epoch 15/2000, Loss: 0.6344410181045532
Epoch 16/2000, Loss: 0.7258164286613464
Epoch 17/2000, Loss: 0.7125737071037292
Epoch 18/2000, Loss: 0.6628161072731018
Epoch 19/2000, Loss: 0.7041367888450623
Epoch 20/2000, Loss: 0.7382788062095642
Epoch 21/2000, Loss: 0.7045692801475525
Epoch 22/2000, Loss: 0.7076061367988586
Epoch 23/2000, Loss: 0.6998800039291382
Epoch 24/2000, Loss: 0.6678525805473328
Epoch 25/2000, Loss: 0.6882390975952148
Epoch 26/